In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from img_prep import prepare_image
import time

In [ ]:
value_list = ['A','B','C','D','E','F','G','H','I','K','L','O']
text= " "
temp=0
previouslabel=None
previousText=" "
label = None
def loadModel(model_path):
    model = load_model(model_path)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
    return model

def predict(img, model):
    try:
        assert img.shape == (1,224,224,3)
    except:
        raise ValueError("Invalid image dimensions")
    
    digits_array = model.predict(img)     
    digits_array = np.squeeze(digits_array)
    digit = np.where(digits_array==np.max(digits_array))[0]
    return digit

def findHand(img):
    hand = img[100:324,100:324]
    hand = cv2.resize(hand, (224,224))
    hand = hand/255
    hand = hand.reshape(1,224,224,3) 
    return hand


if __name__ == "__main__":
    font = cv2.FONT_HERSHEY_SIMPLEX
    model_path = "model.h5"
    model = loadModel(model_path)
    cap = cv2.VideoCapture(0)
    
    while(True):
        ret,frame = cap.read()
        frame = cv2.flip(frame,1)
        cv2.rectangle(frame, (100,100), (300,300), (20,34,255),2)
        hand = findHand(frame)
        pred_index = max(predict(hand, model))
        predictions = model.predict(hand)
        pred = predictions[0].tolist()
        index_pred = pred.index(max(pred))
        label = value_list[index_pred]
        if(label!=None):
            if(temp==0):
                previouslabel=label
            if previouslabel==label :
                previouslabel=label
                temp+=1
            else :
                temp=0
            if(temp==6):
                if(label=='P'):
                
                    label=" "
                text= text + label
                if(label=='Q'):
                    words = re.split(" +",text)
                    words.pop()
                    text = " ".join(words)
                    #text=previousText
                print(text)
                
            
        cv2.putText(frame,str(label), (350,350),font,2,(100,200,25),3, cv2.LINE_AA)
        cv2.putText(frame,text,(50,450), font,3,(0,0,255),2)
        cv2.imshow("Hand Sign Digit", frame)
        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()